Сегментация контекстов и предварительный анализ.

Для удобства я продублировала все составные единицы по группам, составила из них словарь. Также добавила словарь по трех словарям, которые используются MeCab для сегментации контекстов, во избежание путаницы в терминологии назвала их парсерами.

Алгоритм следующий. Идем по словарю всех составных единиц и списку самих единиц для каждого типа. Сначала смотрим, есть ли файл с контекстами по составной единице. Если нет, добавлем единицу в список с такими единицами. Если файл с контекстами есть, открываем его и проводим сегментацию определенным парсером. Затем постепенно добавляем информацию: элементы сегментированной составной единицы (или ее саму), сегментировалась ли она, является ли сегментация (если была) ожидаемой, и есть ли присоединение элементов составной единицы (или ее самой) к соседям по контексту. Сохраняем информацию в файл и переходим к следующему парсеру. После того, как исходный файл с контекстами разобран каждым из парсеров, переходим к следующей единице. И т.д.

In [ ]:
from itertools import pairwise
import pandas as pd
import os, os.path as osp
import re
from tqdm import tqdm

In [ ]:
import MeCab
import unidic
import ipadic
import jumandic

In [ ]:
# составные единицы группы канго с "постфиксами"

kango_with_postfixes = [
    '図書室', '事務室', '研究室', '実験室', '保健室', '新聞室', '美容室', '喫茶室', '調理室', '冷蔵室',
    '密室', '茶室', '教室', '正室', '側室', '客室', '個室', '自室', '病室', '皇室',
    '図書館', '体育館', '映画館', '大使館', '博物館', '公民館', '区民館', '美術館', '記念館', '文書館',
    '休館', '開館', '公館', '旅館', '会館', '閉館', '分館', '本館', '旧館', '別館',
    '研究所', '停留所', '案内所', '事務所', '市役所', '区役所', '保健所', '実験所', '代官所', '保育所',
    '近所', '場所', '役所', '寝所', '別所', '住所', '個所', '札所', '入所', '出所',
    '人事院', '美容院', '四合院', '伝法院', '地理院', '参議院', '大学院', '衆議院', '密蔵院', '感化院',
    '病院', '子院', '入院', '退院', '法院', '寺院', '上院', '転院', '当院', '議院',
    '運動場', '事業場', '飛行場', '野球場', '駐車場', '練習場', '採取場', '駐輪場', '処理場',
    '登場', '会場', '市場', '球場', '上場', '式場', '戦場', '劇場', '開場', '工場', 
    '立場', '職場', '相場', '馬場', '現場', '夏場', '役場', '職場', '地場', '広場',
    '消防士', '弁護士', '保育士', '管理士', '技能士', '税理士', '会計士', '代議士',
    '騎士', '武士', '紳士', '義士', '兵士', '壮士', '浪士', '同士', '藩士', '範士',
    '研究者', '教育者', '分担者', '使用者', '雇用者', '債務者', '労働者', '筆記者', '消費者',
    '役者', '患者', '学者', '他者', '前者', '後者', '業者', '著者', '死者', '筆者',
    '会社員', '銀行員', '組合員', '通報員', '特派員', '作業員', '添乗員', '従業員',
    '一員', '全員', '役員', '委員', '職員', '会員', '船員', '人員', '総員', '行員',
    '小説家', '政治家', '音楽家', '写真家', '専門家', '脚本家', '研究家', '作曲家', '舞踊家', '投資家',
    '作家', '農家', '商家', '画家',
    '調理師', '美容師', '薬剤師', '絵図師', '説教師', '看護師', '宣教師', '連歌師', '助産師', '馬術師',
    '教師', '講師', '医師', '薬師', '牧師', '彫師', '漁師', '恩師', '法師', '老師'
]

In [ ]:
# составные единицы группы канго с "префиксами"

kango_with_prefixes = [
    '無試験', '無計画', '無意味', '無関心', '無気力', '無条件', '無関係', '無防備', '無着陸飛行', '無教養',
    '無理', '無数', '無量', '無念', '無性', '無料', '無駄', '無視', '無効', '無職',
    '不必要', '不注意', '不親切', '不経済', '不勉強', '不自由', '不可能', '不平等', '不正確', '不参加', 
    '不要', '不便', '不調', '不安', '不満', '不明', '不平', '不足', '不和', '不利', '不正',
    '非公式', '非科学', '非論理', '非生産', '非人間', '非整数', '非提携', '非合法', '非産油', '非暴力',
    '非常', '非行', '非道', '非情', '非難', '非番', '非礼', '非核', '非極', '非人',
    '未完全', '未発表', '未使用', '未経験', '未編集', '未成年', '未分離', '未就学', '未曾有', '未完了',
    '未満', '未熟', '未知', '未婚', '未定', '未聞', '未完', '未進', '未然', '未詳',
    '総生産', '総人口', '総収入', '総輸入', '総選挙', '総人口', '総死亡', '総工費', '総領事', '総打数', 
    '総称', '総会', '総合', '総勢', '総数', '総括', '総額', '総員', '総則', '総評',
    '全科目', '全学生', '全生産', '全領域', '全産業', '全国民', '全日本', '全領土', '全人類', '全外貨',
    '全国', '全員', '全階', '全体', '全部', '全権', '全開', '全額', '全集', '全般',
    '諸問題', '諸事情', '諸費用', '諸欲求', '諸能力', '諸官庁', '諸情勢', '諸職人', '諸施策', '諸制度',
    '諸国', '諸島', '諸家', '諸州', '諸仏', '諸点', '諸元', '諸悪', '諸神', '諸法',
    '各会社', '各新聞', '各方面', '各分野', '各事業', '各大学', '各条項', '各学校', '各心理', '各効果',
    '各国', '各地', '各県', '各般', '各年', '各部', '各道', '各種', '各庁', '各月'
]

In [ ]:
# составные глаголы группы ваго, написание которых может варьироваться

wago_verbs_longer = [
    '取り組む', '勝ち越す', '呼び出す', '書き込む', '申し込む', '飛び込む', '差し出す', '振り込む'
]

wago_verbs_shorter = [
    '取組む', '勝越す', '呼出す', '書込む', '申込む', '飛込む', '差出す', '振込む'
]

In [ ]:
# составные существительные группы ваго, написание которых может варьироваться

wago_nouns_longer = [
    '取り組み', '受け付け', '申し込み', '組み立て', '割り引き', '立ち入れ', '売り上げ', '押し売り', '飛び込み', '差し出し'
]

wago_nouns_shorter = [
    '取組み', '受付け', '申込み', '組立て', '割引き', '立入れ', '売上げ', '押売り', '飛込み', '差出し'
]

wago_nouns_shortest = [
    '取組', '受付', '申込', '組立', '割引', '立入', '売上', '押売', '飛込', '差出'
]

In [ ]:
# составные существительные с сочинительной связью

noun_conjuncts = [
    '英独仏', '高中小', '雪月花', '上中下', '松竹梅', '親子孫', '年月日', 
    '父母', '子孫', '大小', '兄弟', '巨大', '姉妹', '危険', '白黒', '善良', '邪悪'
]

In [ ]:
# словарь всех составных единиц: ключи - типы составных единиц, значения - списки самих составных единиц

all_compound_units = {
    'kango_with_postfixes': kango_with_postfixes,
    'kango_with_prefixes': kango_with_prefixes,
    'wago_verbs_longer': wago_verbs_longer,
    'wago_verbs_shorter': wago_verbs_shorter,
    'wago_nouns_longer': wago_nouns_longer,
    'wago_nouns_shorter': wago_nouns_shorter,
    'wago_nouns_shortest': wago_nouns_shortest,
    'noun_conjuncts': noun_conjuncts
}

In [ ]:
# словарь всех парсеров: ключи - названия словарей, значения - сами парсеры (морфологический анализатор MeCab с подключенными словарями)

parsers = {
    'unidic': MeCab.Tagger('-O wakati'),
    'ipadic': MeCab.Tagger(ipadic.MECAB_ARGS + ' -O wakati'),
    'jumandic': MeCab.Tagger(jumandic.MECAB_ARGS + ' -O wakati')
}

In [ ]:
# функция для сегментации, на вход принимает файл с "сырыми" контекстами и парсер

def parse_contexts(raw_contexts, parse_with):
    return raw_contexts['context'].apply(parse_with.parse).str.split()

In [ ]:
# функция, которая выводит элементы сегментированной составной единицы;
# в качестве аргументов функция принимает: сегментированные контексты,
# регулярное выражение для поиска составной единицы, 
# а также максимально возможное количество элементов единицы;
# принцип работы функции: поскольку составная единица может встретиться в контексте несколько раз (а глаголы - и в разной форме), 
# я решила искать их, создав "окошечко" с границами поиска, а найденные совпадения сохранять в список "кандидатов"

def split_units_in_row(segmented_context, compound_unit_re, max_segments):
    candidates = []
    start = 0
    for end in range(1, len(segmented_context) + 1):
        if end - start > max_segments:
            start += 1
        if compound_unit_re.search(''.join(segmented_context[start:end])):
            while compound_unit_re.search(''.join(segmented_context[start:end])):
                start += 1
            candidates.append(segmented_context[start - 1:end])
            start = end
    return candidates

In [ ]:
# функция для проверки сегментации, на вход принимает строку и ожидаемые границы элементов составной единицы

def check_unexpected_segmentation(row, expected_bounds):
    expected_segments = [
        row['compound unit form'][start:end] for start, end in pairwise(expected_bounds)
    ]
    return "NO" if all(
        any(expected_segment in split_unit for split_unit in row['split units'])
        for expected_segment in expected_segments
    ) else "YES"

In [ ]:
# основной код

# список составных единиц, контексты с которыми не были найдены в корпусе BCCWJ в процессе парсинга
no_contexts_found = []

# переберем все составные единицы из созданного выше словаря 
# нужны kind - тип составной единицы (ключ в словаре) и compound_units - сами составные единицы (значения в словаре)
# воспользуемся tqdm для отслеживания процесса
for kind, compound_units in tqdm(all_compound_units.items()):
    # для каждой составной единицы сделаем следующее:
    for compound_unit in compound_units:
        # проверим, есть ли файл с "сырыми" контекстами в нужной папке (папка имеет то же название, что и тип единицы) 
        # если файла нет, записываем единицу в список ненайденных
        if f'{compound_unit}.csv' not in os.listdir(f'{kind}'):
            no_contexts_found.append(compound_unit)
        # если же файл есть, продолжаем работу с ним
        else:
            # прочитаем таблицу, для удобства работы добавим столбец с названием составной единицы и переименуем колонку с контекстами
            contexts = pd.read_csv(f'{kind}/{compound_unit}.csv', sep=',', encoding='utf-8', index_col=0).rename(columns={'0': 'context'})
            contexts.insert(0, 'compound unit', compound_unit)

            # займемся сегментацией с помощью каждого из парсеров
            for parser_name, parse_with in parsers.items():
                # воспользуемся функцией для сегментации файла, добавим стобец со списком сегментированных единиц в контексте
                contexts['segmented'] = parse_contexts(contexts, parse_with)

                
                # сохраним в переменную регулярное выражение для нахождения составной единицы;
                # для глаголов приходится добавлять условие из-за того, что они спрягаются: 
                # последний символ начальной формы может заменяться на один символ хираганы других трех рядов годзюон 
                # или на символ хираганы пятого рядя годзюон + う)
                compound_unit_re = re.compile(
                    f'{compound_unit[:-1]}(?:[^おこごそぞとどのほぼぽもろ\u30A0-\u30FF\u4E00-\u9FFF\u3400-\u4DBF]|[おこごそぞとどのほぼぽもろ]う)' 
                    if 'wago_verbs' in kind 
                    else compound_unit
                )

                # выделим сегментированные элементы составной единицы с помощью функции
                # и добавим в таблицу с контекстами соответствующий столбец               
                contexts['split units'] = contexts.segmented.apply(
                    split_units_in_row,
                    compound_unit_re=compound_unit_re,
                    max_segments=len(compound_unit) + 1
                )

                # в тех случаях, когда в контексте составная единица встретилась несколько раз, 
                # добавим в таблицу строки где вся информация дублируется, а каждый вариант из списка 'split units' на своей строке, 
                # таблицу сохраним
                contexts_exploded = contexts.explode('split units', ignore_index=True)
                
                # в новую таблицу добавим столбец с информацией, сегментировалась ли составная единица
                contexts_exploded['the unit is split'] = contexts_exploded['split units'].map(
                    lambda split_units: 'NO'
                    if len(split_units) == 1
                    else 'YES'
                )

                # найдем форму составной единицы (у глаголов может отличаться от начальной)
                contexts_exploded['compound unit form'] = (
                    contexts_exploded['split units'].apply(lambda units: compound_unit_re.search(''.join(units))[0])
                )

                # укажем ожидаемые границы разбиения составной единицы на элементы (зависят от типа составной единицы):
                # для постфикса ожидаемым будет отделение последнего символа, для префикса - первого и т.д.,
                # главное - не разбивать смысловой "корень"
                if kind == 'kango_with_postfixes':
                    expected_bounds = [0, -1, None]
                elif 'longer' in kind:
                    expected_bounds = [0, 2, None]
                elif kind == 'noun_conjuncts' and len(compound_unit) == 3:
                    expected_bounds = [0, 1, 2, None]
                else:
                    expected_bounds = [0, 1, None]
            
                # воспользуемся функцией для проверки сегментации составных единиц, добавим соответствующую информацию в таблицу
                contexts_exploded['unexpected segmentation'] = contexts_exploded.apply(
                    check_unexpected_segmentation,
                    expected_bounds=expected_bounds,
                    axis=1
                )
              
                # проверим, не присоединились ли наши элементы составной единицы к соседям слева и справа в контексте, 
                # добавим столбец с этой информацией
                contexts_exploded['unexpected joining'] = (
                    contexts_exploded['split units'].str.join('').str.fullmatch(compound_unit_re)
                    .map({True: 'NO', False: 'YES'})
                )
                
                # сохраним итоговую таблицу в файл
                contexts_exploded.to_csv(f'segmentation_analysis/{kind}/using_{parser_name}/{compound_unit}_analysis.csv')